In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document

import os
import dotenv
import yaml

dotenv.load_dotenv()


True

In [15]:
base_path = os.getenv("BASE_PATH", default=".")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", default=None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", default=None)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key = OPENAI_API_KEY
)

vectordb_path = "C://Users/Sese/AI_Study_Record/RAG_AGENT/rag_0608/chroma_db"

vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embeddings,
)

vectorstore.delete_collection()

vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embeddings,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=GOOGLE_API_KEY,
)



In [ ]:
from langchain_text_splitters import HTMLSectionSplitter

headers_to_split_on = [
    ("h1", "Header 1"),
]


html_splitter = HTMLSectionSplitter(
    headers_to_split_on=headers_to_split_on,
)

In [ ]:
documents = []
length ={}
from tqdm import tqdm
for path in os.listdir(base_path):
    filename = os.path.join(base_path, path)
    if int(path.split('_')[0])<=2508:
        continue
    with open(filename, "r", encoding="utf-8") as file:
        html_content = file.read()
    chunks = html_splitter.split_text(html_content)
    # chunks = html_splitter.split_text_from_file(filename)
    
    title = ('_').join(path.split('_')[2:])[:-5]  # Assuming the file name ends with '.html'

    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=f"제목: {title}\n\n{chunk.page_content}",
            metadata={
                "source": filename,
                "chunk_metadata": str(chunk.metadata),
                "chunk_index": i,
                "total_chunks": len(chunks),
            }
        )
        documents.append(doc)
        print(f"{title} ({i+1}/{len(chunks)}) - {len(chunk.page_content)} characters")
        vectorstore.add_documents([doc])

In [34]:
vectorstore_path = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0608\chroma_db"
vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectorstore_path,
    embedding_function=embeddings,
)


# 벡터스토어와 retriever 세팅
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

print(os.getcwd())
with open("queries.yaml", "r", encoding="utf-8") as f:
    queries = yaml.safe_load(f).get("queries", [])

c:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0608


In [35]:
query_log = []

for query in queries:
    docs = retriever.invoke(query)
    print(f"\n🔍 Query: {query}")

    result_entry = {"query": query, "results": []}
    
    for doc in docs:
        source = doc.metadata.get("source", "Unknown")
        excerpt = doc.page_content.replace("\n", " ").strip()
        result_entry["results"].append({
            "source": os.path.basename(source),
            "chunk_index": doc.metadata.get("chunk_index", 99),
            "total_chunks": doc.metadata.get("total_chunks", 99),
            "excerpt": excerpt,
        })
        print(f"  - {source}")

    query_log.append(result_entry)

# 결과 저장
with open("query_log.yaml", "w", encoding="utf-8") as f:
    yaml.dump(query_log, f, allow_unicode=True)
    
    import yaml

with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

lines = ["# 🔍 RAG Query Log\n"]

for entry in log_data:
    query = entry.get("query", "")
    results = entry.get("results", [])

    lines.append(f"\n## 🧪 Query: **{query}**\n")
    lines.append("| Rank | Source | Excerpt |")
    lines.append("|------|--------|---------|")

    for i, result in enumerate(results, 1):
        source = result.get("source", "N/A")
        excerpt = result.get("excerpt", "").replace("|", "\\|")  # Markdown-safe
        lines.append(f"| {i} | `{source}` | {excerpt} |")

# Markdown 파일 저장
with open("query_log.md", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print("✅ query_log.md 생성 완료!")



🔍 Query: 뎅기열과 비슷한 매개체로 인한 감염병은?
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2522_5676_Leptospira interrogans.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2522_5660_Vibrio spp..html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1757_4091_요로감염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2747_5639_염증의 양상.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2521_5678_Pseudomonas aeruginosa.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\2525_5684_모기 매개 감염병.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1643_3848_여행 관련 바이러스 감염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1640_3835_녹농균 감염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1778_4160_위식도역류질환.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1636_3826_발열, 불명열.html

🔍 Query: 불명열 진단 순서를 설명하시오
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1698_3968_석회화 건염.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theory_texts\1698_3967_회전근개파열.html
  - C:\Users\Sese\autosave\알렌 이론 추출\theor

In [5]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain

# ⚠️ 보안 주의: 실서비스나 Git 업로드에는 절대 포함하지 마세요

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=GOOGLE_API_KEY,
)


In [36]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an expert in the field of medicine. Based on the following context, please answer the question.:

Context:
{context}

Question: {question}

Answer:
"""
)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an expert in the field of medicine. Based on the context provided below, analyze all relevant categories and provide a comprehensive answer. Do not limit your answer to only directly similar examples—include all entries that fall under the same classification or grouping where appropriate.

Context:
{context}

Question: {question}

Answer:
"""
)


prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 의학 분야의 전문가입니다. 아래에 제공된 문맥을 바탕으로, 관련된 모든 범주를 분석하고 포괄적인 답변을 제시하십시오. 질문에 직접적으로 유사한 예에만 국한하지 말고, 같은 분류나 범주에 해당하는 항목들을 모두 포함하도록 하십시오.

문맥:
{context}

질문: {question}

답변:
"""
)


In [39]:
readme_lines = ["# LLM Query Results\n"]

with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

for entry in log_data:
    readme_lines.append(f"\n## 🧪 Query: **{entry['query']}**\n")
    readme_lines.append("| Rank | Source | chunk | 평가 ")
    readme_lines.append("|------|--------|---------|---------|")
    for i, result in enumerate(entry["results"], 1):
        source = result["source"]
        total_chunks = result["total_chunks"]
        chunk_index = result["chunk_index"]
        readme_lines.append(f"| {i} | `{source}` | {chunk_index+1}/{total_chunks} |  |")
with open("README.md", "w", encoding="utf-8") as f:
    f.write("\n".join(readme_lines))

In [40]:
augmented_log = []
markdown_lines = ["# LLM Query Results\n"]

with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

for entry in log_data:
    query = entry.get("query", "")
    results = entry.get("results", [])
    contents = [i['excerpt'] for i in results]
    context = "\n".join([f"{'#'*10}\n{i+1}. {content}" for i, content in enumerate(contents)])
    

    llm_chain = prompt_template | llm
    response = llm_chain.invoke({"context": context, "question": query}).content.strip()

    # 로그에 응답 추가
    entry["llm_answer"] = response
    augmented_log.append(entry)

    # Markdown에 포맷 추가
    markdown_lines.append(f"## 🔍 Query\n{query}\n")
    markdown_lines.append("### 📚 Context\n")
    for idx, result in enumerate(results):
        markdown_lines.append(f"<details>\n<summary>{idx+1}. {result['source']} ({result['chunk_index']+1}/{result['total_chunks']})</summary>\n")
        markdown_lines.append(f"{result['excerpt']}\n")
        markdown_lines.append("</details>\n")
    markdown_lines.append(f"### 💬 Answer\n{response}\n")
    markdown_lines.append("---\n")

# YAML로 저장
with open("[result] LLM_generated.yaml", "w", encoding="utf-8") as f:
    yaml.dump(augmented_log, f, allow_unicode=True, sort_keys=False)

# Markdown으로 저장
with open("[result] LLM_generated.md", "w", encoding="utf-8") as f:
    f.write("\n".join(markdown_lines))

print("✅ Results saved")

✅ Results saved
